In [1]:
import matplotlib.pyplot as plt
import numpy as np
import copy
from scipy.stats import beta
import matplotlib
import pandas as pd
from models import BetaBernoulli

matplotlib.rcParams['font.size'] = 8
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['text.usetex'] = False

num_samples = 1000

from models import BetaBernoulli, SumOfBetaEce
from data_utils import datafile_dict, datasize_dict, num_classes_dict
from active_utils import prepare_data

In [2]:
datafile = datafile_dict['cifar100']
categories, observations, confidences, idx2category, category2idx = prepare_data(datafile, False)

df = pd.read_csv("../data/cifar100/predictions.txt", sep='\s') 
id2class = dict(enumerate(set(df['Correct'])))
class2id = {id2class[k]: k for k in id2class}
print(id2class)

Accuracy: 0.741
{0: 'bottle', 1: 'sweet_pepper', 2: 'crocodile', 3: 'beetle', 4: 'snake', 5: 'house', 6: 'streetcar', 7: 'television', 8: 'bus', 9: 'woman', 10: 'willow_tree', 11: 'hamster', 12: 'lamp', 13: 'rocket', 14: 'can', 15: 'forest', 16: 'turtle', 17: 'train', 18: 'chair', 19: 'butterfly', 20: 'maple_tree', 21: 'orchid', 22: 'cockroach', 23: 'baby', 24: 'lizard', 25: 'snail', 26: 'wardrobe', 27: 'lobster', 28: 'poppy', 29: 'table', 30: 'cloud', 31: 'tiger', 32: 'skyscraper', 33: 'crab', 34: 'boy', 35: 'man', 36: 'pine_tree', 37: 'rabbit', 38: 'caterpillar', 39: 'bridge', 40: 'couch', 41: 'aquarium_fish', 42: 'fox', 43: 'mouse', 44: 'dolphin', 45: 'worm', 46: 'tulip', 47: 'elephant', 48: 'lion', 49: 'camel', 50: 'shark', 51: 'flatfish', 52: 'bee', 53: 'road', 54: 'cattle', 55: 'trout', 56: 'keyboard', 57: 'kangaroo', 58: 'porcupine', 59: 'pear', 60: 'orange', 61: 'leopard', 62: 'bear', 63: 'squirrel', 64: 'mountain', 65: 'shrew', 66: 'sea', 67: 'pickup_truck', 68: 'clock', 69: '

/Users/disiji/Dropbox/current/bayesian-blackbox/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [3]:
# accuracy models
accuracy_model = copy.deepcopy(BetaBernoulli(k=100, prior=None))
accuracy_model.update_batch(categories, observations)

# ece models for each class
ece_models = [copy.deepcopy(SumOfBetaEce(num_bins=10, weight=None, prior_alpha=None, prior_beta=None)) for i in range(100)]
for class_idx in range(100):
    mask_idx = [i for i in range(len(observations)) if observations[i] == class_idx]
    observations_sublist = [observations[i] for i in mask_idx]
    confidences_sublist = [confidences[i] for i in mask_idx]
    ece_models[class_idx].update_batch(confidences_sublist, observations_sublist)

In [4]:
# draw samples from posterior of classwise accuracy
accuracy_samples = accuracy_model.sample(num_samples) # (num_categories, num_samples)
ece_samples = np.array([ece_models[class_idx].sample(num_samples) for class_idx in range(100)]).squeeze() # (num_categories, num_samples)

In [9]:
# plot